In [1]:
import pandas as pd
from stage import fullPrint
from datetime import date
import yahoo_fin.stock_info as yf
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)

In [2]:
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = (nasdaqList + nyseList)

In [3]:
SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"}
spList = yf.tickers_sp500()
spList = set(spList)
dictionary = {}
i = 0
allLen = str(len(nasdaqList))
for ticker in nasdaqList:
    try:
        df = yf.get_data(ticker, start_date="1995-01-02",end_date="2022-01-17", index_as_date = True, interval="1wk")
        df.to_pickle('stockData/nasdaq/'+ticker+'.pkl')
        industry = yf.get_company_info(ticker).at['sector','Value']
        dictionary[ticker] = SectorDict[industry]
    except:
        print(ticker+" not found")
    i = i+1
    print(str(i)+'/'+allLen)
import pickle
try:
    sectorDict = open('stockData/nasdaq.pkl', 'wb')
    pickle.dump(dictionary, sectorDict)
    sectorDict.close()
  
except:
    print("Something went wrong")

AACG not found
1/5560
AACI not found
2/5560
AACIU not found
3/5560
AACIW not found
4/5560
AADI not found
5/5560
AADR not found
6/5560
AAL not found
7/5560
AAME not found
8/5560
AAOI not found
9/5560
AAON not found
10/5560
AAPL not found
11/5560
AATC not found
12/5560
AAWW not found
13/5560
AAXJ not found
14/5560
ABCB not found
15/5560
ABCL not found
16/5560
ABCM not found
17/5560
ABEO not found
18/5560
ABGI not found
19/5560
ABIO not found
20/5560
ABMD not found
21/5560
ABNB not found
22/5560
ABOS not found
23/5560
ABSI not found
24/5560
ABST not found
25/5560
ABTX not found
26/5560
ABUS not found
27/5560
ABVC not found
28/5560
ACABU not found
29/5560
ACAD not found
30/5560
ACAH not found
31/5560
ACAHU not found
32/5560
ACAHW not found
33/5560
ACAXU not found
34/5560
ACB not found
35/5560
ACBA not found
36/5560
ACBAU not found
37/5560
ACBAW not found
38/5560
ACBI not found
39/5560
ACCD not found
40/5560
ACER not found
41/5560
ACET not found
42/5560
ACEV not found
43/5560
ACEVU not foun

In [3]:
import numpy as np
import yahoo_fin.stock_info as yf
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spList = yf.tickers_sp500()
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,4]

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,0]/df.iat[index-30,0]
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,0]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,0])


In [4]:
from stage import fullPrint
import swifter
import yahoo_fin.stock_info as yf
import pandas as pd
import os
spDf = yf.get_data('^GSPC', start_date="1995-01-02",end_date="2022-01-17", index_as_date = True, interval="1wk")
for ticker in allList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        if(stockDf['close'].isnull().values.any()):
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            continue
        stockDf['fiveYearHigh'] = stockDf['close'].rolling(window=260).max().shift(1).fillna(0)  
        stockDf['WMA30'] = stockDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['volumePerc'] = stockDf['volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = 0
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, 3] = calculatePeak(stockDf, numIndex, 260, 3)
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        stockDf['resistance'] = 0
        print(ticker)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 25344.37it/s]


AACG


Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 25182.06it/s]


AACI


Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 25064.75it/s]


...

ZUO


Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 12505.65it/s]


ZWS


Pandas Apply: 100%|██████████| 1411/1411 [00:00<00:00, 11305.55it/s]


ZYME


In [ ]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-02",end_date="2022-01-17", index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [ ]:
pd.read_pickle('stockData/nyseNasdaq/AAPL.pkl').to_csv("AAPL.csv")

In [ ]:
for ticker in allList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        stockDf = stockDf.drop(['open','high','low','adjclose'],axis=1)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass